In [3]:
#check the direction

In [4]:
import os
print(os.getcwd())

C:\Program Files\Microsoft VS Code


In [5]:
import os

# Change to your project directory (update the path as needed)
os.chdir(r"H:\Regional_Water_Company\For_WR_Report\New_Geo_Data")

# Verify the change
print(os.getcwd())

H:\Regional_Water_Company\For_WR_Report\New_Geo_Data


In [4]:
import pandas as pd

# Read the Excel file
Geo_Data = pd.read_excel("New_Merge_Data_dd.xlsx")
# Show the first 5 rows
print(Geo_Data.head())

                 point_name  Depth_1  Depth_2 Type_Geo Elevation  Total_Depth  \
0  abasabad tokhmanjir asma      0.0      3.0       GC      1057         26.6   
1  abasabad tokhmanjir asma      3.0      6.6       GP      1057         26.6   
2  abasabad tokhmanjir asma      6.6      9.4       CL      1057         26.6   
3  abasabad tokhmanjir asma      9.4     14.0       GP      1057         26.6   
4  abasabad tokhmanjir asma     14.0     15.5       CL      1057         26.6   

    Lon_D    Lat_D  Thickness_meters  Thickness_Percent  
0  59.429  36.5557               3.0          11.278195  
1  59.429  36.5557               3.6          13.533835  
2  59.429  36.5557               2.8          10.526316  
3  59.429  36.5557               4.6          17.293233  
4  59.429  36.5557               1.5           5.639098  


In [7]:
import pandas as pd

# Read the geological data
Geo_Data = pd.read_excel("New_Merge_Data_dd.xlsx")

# Define the geological layer abbreviations, full names, and categories
layer_definitions = {
    'GC': {'full_name': 'Gravelly Clay', 'category': 'Fine-grained'},
    'GP': {'full_name': 'Poorly Graded Gravel', 'category': 'Coarse-grained'},
    'CL': {'full_name': 'Clay (Low Plasticity)', 'category': 'Fine-grained'},
    'Undifind': {'full_name': 'Undefined', 'category': 'Uncategorized'},
    'GW': {'full_name': 'Well-Graded Gravel', 'category': 'Coarse-grained'},
    'SM': {'full_name': 'Silty Sand', 'category': 'Coarse-grained'},
    'CL-ML': {'full_name': 'Clay-Silt Mixture', 'category': 'Fine-grained'},
    'BEDROCK': {'full_name': 'Bedrock', 'category': 'Uncategorized'},
    'SP': {'full_name': 'Poorly Graded Sand', 'category': 'Coarse-grained'},
    'SW': {'full_name': 'Well-Graded Sand', 'category': 'Coarse-grained'},
    'ML': {'full_name': 'Silt (Low Plasticity)', 'category': 'Fine-grained'},
    'OL': {'full_name': 'Organic Silt/Clay', 'category': 'Fine-grained'},
    'SC': {'full_name': 'Clayey Sand', 'category': 'Coarse-grained'},
    'CCL': {'full_name': 'Clay (Low Plasticity)', 'category': 'Fine-grained'},
    'CH': {'full_name': 'Clay (High Plasticity)', 'category': 'Fine-grained'},
    'GM': {'full_name': 'Gravelly Silt', 'category': 'Fine-grained'},
    'GP-GM': {'full_name': 'Gravel-Silt Mixture', 'category': 'Medium'},
    'SC-SM': {'full_name': 'Sand-Silt Mixture', 'category': 'Medium'},
    '<Null>': {'full_name': 'Missing Data', 'category': 'Uncategorized'}
}

# Function to print layer definitions by category
def print_layer_categories():
    print("Geological Layer Classifications:\n")
    for category in ['Coarse-grained', 'Fine-grained', 'Medium', 'Uncategorized']:
        print(f"{category} Layers:")
        for abbr, info in layer_definitions.items():
            if info['category'] == category:
                print(f"  {abbr}: {info['full_name']}")
        print()

# Print the categorized layers
print_layer_categories()

# Add full names and categories to the DataFrame
Geo_Data['Full_Name'] = Geo_Data['Type_Geo'].map(lambda x: layer_definitions.get(x, {'full_name': 'Unknown'})['full_name'])
Geo_Data['Category'] = Geo_Data['Type_Geo'].map(lambda x: layer_definitions.get(x, {'category': 'Uncategorized'})['category'])

# Calculate thickness sum per point and add validation columns
Geo_Data['Thickness_Sum'] = Geo_Data.groupby('point_name')['Thickness_Percent'].transform('sum')
Geo_Data['Thickness_Difference'] = Geo_Data['Thickness_Sum'] - 100
Geo_Data['Thickness_Status'] = Geo_Data['Thickness_Sum'].apply(
    lambda x: 1 if x > 100 else (-1 if x < 100 else 0)
)

# Analysis: Count unique points per category
point_category_counts = Geo_Data.groupby('Category')['point_name'].nunique()
print("\nNumber of Unique Points per Category:")
for category, count in point_category_counts.items():
    print(f"  {category}: {count}")

# Analysis: Thickness_Status distribution
thickness_status_counts = Geo_Data.groupby('Thickness_Status')['point_name'].nunique()
print("\nThickness Status Distribution (Unique Points):")
print(f"  Sum < 100 (Status -1): {thickness_status_counts.get(-1, 0)}")
print(f"  Sum = 100 (Status 0): {thickness_status_counts.get(0, 0)}")
print(f"  Sum > 100 (Status 1): {thickness_status_counts.get(1, 0)}")

# Analysis: Thickness_Percent statistics by category
thickness_stats = Geo_Data.groupby('Category')['Thickness_Percent'].describe()
print("\nThickness Percent Statistics by Category:")
print(thickness_stats)

# Filter out points where the absolute Thickness_Difference is greater than 1
Filtered_Geo_Data = Geo_Data[Geo_Data['Thickness_Difference'].abs() <= 1]
filtered_points = Filtered_Geo_Data['point_name'].nunique()

# Save the complete and filtered DataFrames to Excel
Geo_Data.to_excel("New_Merge_Data_with_Layers.xlsx", index=False)
Filtered_Geo_Data.to_excel("New_Merge_Data_with_Layers_Filtered.xlsx", index=False)

# Display a summary of the data
print("\nSample of Updated DataFrame (Original):")
print(Geo_Data[['point_name', 'Type_Geo', 'Full_Name', 'Category', 'Thickness_Percent', 
                'Thickness_Sum', 'Thickness_Difference', 'Thickness_Status']].head())
print("\nSample of Filtered DataFrame (|Thickness_Difference| <= 1):")
print(Filtered_Geo_Data[['point_name', 'Type_Geo', 'Full_Name', 'Category', 'Thickness_Percent', 
                        'Thickness_Sum', 'Thickness_Difference', 'Thickness_Status']].head())

# Analysis: Number of points filtered out
original_points = Geo_Data['point_name'].nunique()
print(f"\nPoints Analysis:")
print(f"  Total unique points in original data: {original_points}")
print(f"  Total unique points in filtered data: {filtered_points}")
print(f"  Points removed (|Thickness_Difference| > 1): {original_points - filtered_points}")

Geological Layer Classifications:

Coarse-grained Layers:
  GP: Poorly Graded Gravel
  GW: Well-Graded Gravel
  SM: Silty Sand
  SP: Poorly Graded Sand
  SW: Well-Graded Sand
  SC: Clayey Sand

Fine-grained Layers:
  GC: Gravelly Clay
  CL: Clay (Low Plasticity)
  CL-ML: Clay-Silt Mixture
  ML: Silt (Low Plasticity)
  OL: Organic Silt/Clay
  CCL: Clay (Low Plasticity)
  CH: Clay (High Plasticity)
  GM: Gravelly Silt

Medium Layers:
  GP-GM: Gravel-Silt Mixture
  SC-SM: Sand-Silt Mixture

Uncategorized Layers:
  Undifind: Undefined
  BEDROCK: Bedrock
  <Null>: Missing Data


Number of Unique Points per Category:
  Coarse-grained: 381
  Fine-grained: 446
  Medium: 3
  Uncategorized: 249

Thickness Status Distribution (Unique Points):
  Sum < 100 (Status -1): 25
  Sum = 100 (Status 0): 431
  Sum > 100 (Status 1): 18

Thickness Percent Statistics by Category:
                 count       mean        std       min       25%        50%  \
Category                                             

In [ ]:
## To classify the given soil types into three categories: Fine, Medium, and Coarse, along with Uncategorized
## To classify the given soil types into three categories: Fine, Medium, and Coarse, along with Uncategorized
## To classify the given soil types into three categories: Fine, Medium, and Coarse, along with Uncategorized


import pandas as pd

# Read the geological data
Geo_Data = pd.read_excel("New_Merge_Data_dd.xlsx")

# Define the geological layer abbreviations, full names, and categories
layer_definitions = {
    'GC': {'full_name': 'Gravelly Clay', 'category': 'Coarse'},
    'GP': {'full_name': 'Poorly Graded Gravel', 'category': 'Coarse'},
    'CL': {'full_name': 'Clay (Low Plasticity)', 'category': 'Fine'},
    'Undifind': {'full_name': 'Undefined', 'category': 'Uncategorized'},
    'GW': {'full_name': 'Well-Graded Gravel', 'category': 'Coarse'},
    'SM': {'full_name': 'Silty Sand', 'category': 'Medium'},
    'CL-ML': {'full_name': 'Clay-Silt Mixture', 'category': 'Fine'},
    'BEDROCK': {'full_name': 'Bedrock', 'category': 'BED'},
    'SP': {'full_name': 'Poorly Graded Sand', 'category': 'Medium'},
    'SW': {'full_name': 'Well-Graded Sand', 'category': 'Medium'},
    'ML': {'full_name': 'Silt (Low Plasticity)', 'category': 'Fine'},
    'OL': {'full_name': 'Organic Silt/Clay', 'category': 'Fine'},
    'SC': {'full_name': 'Clayey Sand', 'category': 'Medium'},
    'CCL': {'full_name': 'Clay (Low Plasticity)', 'category': 'Fine'},
    'CH': {'full_name': 'Clay (High Plasticity)', 'category': 'Fine'},
    'GM': {'full_name': 'Gravelly Silt', 'category': 'Coarse'},
    'GP-GM': {'full_name': 'Gravel-Silt Mixture', 'category': 'Coarse'},
    'SC-SM': {'full_name': 'Sand-Silt Mixture', 'category': 'Medium'},
    '<Null>': {'full_name': 'Missing Data', 'category': 'Uncategorized'}
}

# Function to print layer definitions by category
def print_layer_categories():
    print("Geological Layer Classifications:\n")
    for category in ['Coarse-grained', 'Fine-grained', 'Medium', 'Uncategorized']:
        print(f"{category} Layers:")
        for abbr, info in layer_definitions.items():
            if info['category'] == category:
                print(f"  {abbr}: {info['full_name']}")
        print()

# Print the categorized layers
print_layer_categories()

# Add full names and categories to the DataFrame
Geo_Data['Full_Name'] = Geo_Data['Type_Geo'].map(lambda x: layer_definitions.get(x, {'full_name': 'Unknown'})['full_name'])
Geo_Data['Category'] = Geo_Data['Type_Geo'].map(lambda x: layer_definitions.get(x, {'category': 'Uncategorized'})['category'])

# Calculate thickness sum per point and add validation columns
Geo_Data['Thickness_Sum'] = Geo_Data.groupby('point_name')['Thickness_Percent'].transform('sum')
Geo_Data['Thickness_Difference'] = Geo_Data['Thickness_Sum'] - 100
Geo_Data['Thickness_Status'] = Geo_Data['Thickness_Sum'].apply(
    lambda x: 1 if x > 100 else (-1 if x < 100 else 0)
)

# Analysis: Count unique points per category
point_category_counts = Geo_Data.groupby('Category')['point_name'].nunique()
print("\nNumber of Unique Points per Category:")
for category, count in point_category_counts.items():
    print(f"  {category}: {count}")

# Analysis: Thickness_Status distribution
thickness_status_counts = Geo_Data.groupby('Thickness_Status')['point_name'].nunique()
print("\nThickness Status Distribution (Unique Points):")
print(f"  Sum < 100 (Status -1): {thickness_status_counts.get(-1, 0)}")
print(f"  Sum = 100 (Status 0): {thickness_status_counts.get(0, 0)}")
print(f"  Sum > 100 (Status 1): {thickness_status_counts.get(1, 0)}")

# Analysis: Thickness_Percent statistics by category
thickness_stats = Geo_Data.groupby('Category')['Thickness_Percent'].describe()
print("\nThickness Percent Statistics by Category:")
print(thickness_stats)

# Filter out points where the absolute Thickness_Difference is greater than 1
Filtered_Geo_Data = Geo_Data[Geo_Data['Thickness_Difference'].abs() <= 1]
filtered_points = Filtered_Geo_Data['point_name'].nunique()

# Save the complete and filtered DataFrames to Excel
Geo_Data.to_excel("New_Merge_Data_with_Layers_4Categories.xlsx", index=False)
Filtered_Geo_Data.to_excel("New_Merge_Data_with_Layers_Filtered_4Categories.xlsx", index=False)

# Display a summary of the data
print("\nSample of Updated DataFrame (Original):")
print(Geo_Data[['point_name', 'Type_Geo', 'Full_Name', 'Category', 'Thickness_Percent', 
                'Thickness_Sum', 'Thickness_Difference', 'Thickness_Status']].head())
print("\nSample of Filtered DataFrame (|Thickness_Difference| <= 1):")
print(Filtered_Geo_Data[['point_name', 'Type_Geo', 'Full_Name', 'Category', 'Thickness_Percent', 
                        'Thickness_Sum', 'Thickness_Difference', 'Thickness_Status']].head())

# Analysis: Number of points filtered out
original_points = Geo_Data['point_name'].nunique()
print(f"\nPoints Analysis:")
print(f"  Total unique points in original data: {original_points}")
print(f"  Total unique points in filtered data: {filtered_points}")
print(f"  Points removed (|Thickness_Difference| > 1): {original_points - filtered_points}")

Geological Layer Classifications:

Coarse-grained Layers:

Fine-grained Layers:

Medium Layers:
  SM: Silty Sand
  SP: Poorly Graded Sand
  SW: Well-Graded Sand
  SC: Clayey Sand
  SC-SM: Sand-Silt Mixture

Uncategorized Layers:
  Undifind: Undefined
  <Null>: Missing Data


Number of Unique Points per Category:
  BED: 23
  Coarse: 412
  Fine: 337
  Medium: 182
  Uncategorized: 231

Thickness Status Distribution (Unique Points):
  Sum < 100 (Status -1): 25
  Sum = 100 (Status 0): 431
  Sum > 100 (Status 1): 18

Thickness Percent Statistics by Category:
                count       mean        std       min       25%        50%  \
Category                                                                     
BED              23.0  13.894067  10.214882  0.555556  7.035330  13.793103   
Coarse         1244.0  14.797436  14.986441  0.400000  5.084746  10.000000   
Fine           1206.0  15.130547  15.404152  0.403551  5.263158  10.000000   
Medium          312.0  13.091880  12.825346  0.7142

In [9]:
import pandas as pd
import rasterio
from rasterio.transform import rowcol
import numpy as np

# File paths
tiff_path_2024 = "WL2024.tif"
tiff_path_1992 = "WL1992.tif"
excel_path = "New_Merge_Data_with_Layers_Filtered_4Categories.xlsx"
output_excel_path = "with_Water_Levels_1992_2024.xlsx"

try:
    # Load the Excel file
    df = pd.read_excel(excel_path)

    # Initialize lists for water levels and problematic rows
    water_levels_2024 = []
    water_levels_1992 = []
    problematic_rows = []

    # Open the 2024 GeoTIFF file
    with rasterio.open(tiff_path_2024) as src_2024, rasterio.open(tiff_path_1992) as src_1992:
        # Get transforms and data
        transform_2024 = src_2024.transform
        raster_data_2024 = src_2024.read(1)
        nodata_2024 = src_2024.nodata if src_2024.nodata is not None else np.nan

        transform_1992 = src_1992.transform
        raster_data_1992 = src_1992.read(1)
        nodata_1992 = src_1992.nodata if src_1992.nodata is not None else np.nan

        # Iterate through each row
        for idx, (lon, lat, point_name) in enumerate(zip(df['Lon_D'], df['Lat_D'], df['point_name'])):
            # Check for invalid or missing coordinates
            if pd.isna(lon) or pd.isna(lat) or str(lon).lower() == '<null>' or str(lat).lower() == '<null>':
                problematic_rows.append((idx, point_name, lon, lat, 'Invalid or missing coordinates'))
                water_levels_2024.append(np.nan)
                water_levels_1992.append(np.nan)
                continue

            try:
                # Convert to float and validate
                lon = float(lon)
                lat = float(lat)

                # Process 2024 data
                row_2024, col_2024 = rowcol(transform_2024, lon, lat)
                if 0 <= row_2024 < raster_data_2024.shape[0] and 0 <= col_2024 < raster_data_2024.shape[1]:
                    value_2024 = raster_data_2024[row_2024, col_2024]
                    water_levels_2024.append(np.nan if value_2024 == nodata_2024 or np.isnan(value_2024) else value_2024)
                else:
                    water_levels_2024.append(np.nan)
                    problematic_rows.append((idx, point_name, lon, lat, 'Coordinates outside 2024 raster bounds'))

                # Process 1992 data
                row_1992, col_1992 = rowcol(transform_1992, lon, lat)
                if 0 <= row_1992 < raster_data_1992.shape[0] and 0 <= col_1992 < raster_data_1992.shape[1]:
                    value_1992 = raster_data_1992[row_1992, col_1992]
                    water_levels_1992.append(np.nan if value_1992 == nodata_1992 or np.isnan(value_1992) else value_1992)
                else:
                    water_levels_1992.append(np.nan)
                    problematic_rows.append((idx, point_name, lon, lat, 'Coordinates outside 1992 raster bounds'))

            except (ValueError, TypeError) as e:
                problematic_rows.append((idx, point_name, lon, lat, f'Error processing coordinates: {str(e)}'))
                water_levels_2024.append(np.nan)
                water_levels_1992.append(np.nan)

    # Add water level columns to the DataFrame
    df['WL2024'] = water_levels_2024
    df['WL1992'] = water_levels_1992

    # Save the updated DataFrame
    df.to_excel(output_excel_path, index=False)
    print(f"Updated Excel file saved to: {output_excel_path}")

    # Display a sample of the updated DataFrame
    print("\nSample of Updated DataFrame with Water Levels:")
    print(df[['point_name', 'Lon_D', 'Lat_D', 'WL2024', 'WL1992']].head())

    # Report problematic rows
    if problematic_rows:
        print("\nRows with Invalid or Out-of-Bounds Coordinates:")
        for idx, point_name, lon, lat, issue in problematic_rows:
            print(f"Row {idx}: Point = {point_name}, Lon = {lon}, Lat = {lat}, Issue = {issue}")

except FileNotFoundError as e:
    print(f"Error: File not found. Check paths: {tiff_path_2024}, {tiff_path_1992}, {excel_path}. Details: {str(e)}")
except Exception as e:
    print(f"Error: Failed to process files. Details: {str(e)}")

Updated Excel file saved to: with_Water_Levels_1992_2024.xlsx

Sample of Updated DataFrame with Water Levels:
                 point_name   Lon_D    Lat_D     WL2024     WL1992
0  abasabad tokhmanjir asma  59.429  36.5557  72.269569  42.360306
1  abasabad tokhmanjir asma  59.429  36.5557  72.269569  42.360306
2  abasabad tokhmanjir asma  59.429  36.5557  72.269569  42.360306
3  abasabad tokhmanjir asma  59.429  36.5557  72.269569  42.360306
4  abasabad tokhmanjir asma  59.429  36.5557  72.269569  42.360306

Rows with Invalid or Out-of-Bounds Coordinates:
Row 1383: Point = joeh paien4 asma, Lon = <Null>, Lat = <Null>, Issue = Invalid or missing coordinates
Row 1384: Point = joeh paien4 asma, Lon = <Null>, Lat = <Null>, Issue = Invalid or missing coordinates
Row 1385: Point = joeh paien4 asma, Lon = <Null>, Lat = <Null>, Issue = Invalid or missing coordinates
Row 1386: Point = joeh paien4 asma, Lon = <Null>, Lat = <Null>, Issue = Invalid or missing coordinates
Row 1387: Point = joeh paie

In [10]:
### for two water depth 1992 to 2024 exactly in this distance 1922 and 2024


import pandas as pd
import numpy as np

# File paths
input_excel_path = "with_Water_Levels_1992_2024.xlsx"
output_excel_path = "Water_Level_1992_2024_Overlap.xlsx"

try:
    # Load the Excel file
    df = pd.read_excel(input_excel_path)

    # Initialize a list to store indices of rows to keep and problematic rows
    rows_to_keep = []
    problematic_rows = []

    # Filter rows where the layer depth range overlaps with the water level range
    for idx, row in df.iterrows():
        wl_2024 = row['WL2024']
        wl_1992 = row['WL1992']
        depth_1 = row['Depth_1']
        depth_2 = row['Depth_2']
        point_name = row['point_name']

        # Check if water levels are valid (not NaN or <Null>)
        if (pd.isna(wl_2024) or str(wl_2024).lower() == '<null>' or 
            pd.isna(wl_1992) or str(wl_1992).lower() == '<null>'):
            problematic_rows.append((idx, point_name, wl_1992, wl_2024, 'Invalid or missing water level data'))
            continue

        # Calculate min and max water levels
        min_wl = min(wl_2024, wl_1992)
        max_wl = max(wl_2024, wl_1992)

        # Keep row if layer depth range overlaps with water level range
        if depth_1 <= max_wl and depth_2 >= min_wl:
            rows_to_keep.append(idx)

    # Create filtered DataFrame
    filtered_df = df.loc[rows_to_keep].copy()

    # Save the filtered DataFrame to a new Excel file
    filtered_df.to_excel(output_excel_path, index=False)
    print(f"Filtered Excel file saved to: {output_excel_path}")
    print(f"Original number of rows: {len(df)}")
    print(f"Filtered number of rows: {len(filtered_df)}")

    # Display a sample of the filtered DataFrame
    print("\nSample of Filtered DataFrame (Layers Overlapping WL1992 to WL2024):")
    print(filtered_df[['point_name', 'Depth_1', 'Depth_2', 'Type_Geo', 'WL1992', 'WL2024']].head())

    # Report problematic rows
    if problematic_rows:
        print("\nRows with Invalid or Missing Water Level Data:")
        for idx, point_name, wl_1992, wl_2024, issue in problematic_rows:
            print(f"Row {idx}: Point = {point_name}, WL1992 = {wl_1992}, WL2024 = {wl_2024}, Issue = {issue}")

except FileNotFoundError as e:
    print(f"Error: File not found. Check path: {input_excel_path}. Details: {str(e)}")
except Exception as e:
    print(f"Error: Failed to process file. Details: {str(e)}")

Filtered Excel file saved to: Water_Level_1992_2024_Overlap.xlsx
Original number of rows: 3035
Filtered number of rows: 906

Sample of Filtered DataFrame (Layers Overlapping WL1992 to WL2024):
           point_name  Depth_1  Depth_2 Type_Geo     WL1992     WL2024
8       abaskhan asma      5.0     10.0       CL   9.874941  23.229906
9       abaskhan asma     10.0     20.0       GW   9.874941  23.229906
10      abaskhan asma     20.0     25.0       CL   9.874941  23.229906
15  abfaakharnakhrisi     22.0     70.0       GC  47.912453  58.952065
20       abfaandishe3     60.0     90.0       GC  69.350441  87.593704

Rows with Invalid or Missing Water Level Data:
Row 1375: Point = joeh paien3 asma, WL1992 = nan, WL2024 = nan, Issue = Invalid or missing water level data
Row 1376: Point = joeh paien3 asma, WL1992 = nan, WL2024 = nan, Issue = Invalid or missing water level data
Row 1377: Point = joeh paien3 asma, WL1992 = nan, WL2024 = nan, Issue = Invalid or missing water level data
Row 1378:

In [12]:
# Replace the Total_Depth column for each point with the corresponding Water_Level_Depth value
# Replace the Total_Depth column for each point with the corresponding Water_Level_Depth value
# Replace the Total_Depth column for each point with the corresponding Water_Level_Depth value


import pandas as pd
import numpy as np

# File paths
input_excel_path = "Water_Level_1992_2024_Overlap.xlsx"
output_excel_path = "Data_Updated_Depths_Total_Depth.xlsx"

try:
    # Load the Excel file
    df = pd.read_excel(input_excel_path)

    # Initialize lists for problematic rows
    problematic_rows = []

    # Group by point_name to ensure consistent Total_Depth per point
    grouped = df.groupby('point_name')

    # Function to update Total_Depth and recalculate Thickness columns
    def update_point_group(group):
        # Get the first valid Water_Level_Depth (should be same for all rows in group)
        water_level = group['Water_Level_Depth'].dropna()
        if water_level.empty or str(water_level.iloc[0]).lower() == '<null>':
            problematic_rows.append((group.name, 'Missing or invalid Water_Level_Depth'))
            return None  # Skip group with no valid water level

        water_level = float(water_level.iloc[0])
        if water_level == 0:
            problematic_rows.append((group.name, 'Water_Level_Depth is zero'))
            return None  # Skip to avoid division by zero

        # Update Total_Depth
        group['Total_Depth'] = water_level

        # Recalculate Thickness_meters
        group['Thickness_meters'] = group['Depth_2'] - group['Depth_1']

        # Recalculate Thickness_Percent
        group['Thickness_Percent'] = (group['Thickness_meters'] / group['Total_Depth']) * 100

        return group

    # Apply updates to each group
    updated_df = grouped.apply(update_point_group).reset_index(drop=True)

    # Remove any None groups (those with invalid water levels)
    if updated_df is not None:
        updated_df = updated_df.dropna(subset=['Total_Depth'])

    # Save the updated DataFrame
    updated_df.to_excel(output_excel_path, index=False)
    print(f"Updated Excel file saved to: {output_excel_path}")
    print(f"Original number of rows: {len(df)}")
    print(f"Updated number of rows: {len(updated_df)}")

    # Display a sample of the updated DataFrame
    print("\nSample of Updated DataFrame:")
    print(updated_df[['point_name', 'Depth_1', 'Depth_2', 'Type_Geo', 'Total_Depth', 
                     'Thickness_meters', 'Thickness_Percent', 'Water_Level_Depth']].head())

    # Report problematic points
    if problematic_rows:
        print("\nProblematic Points:")
        for point_name, issue in problematic_rows:
            print(f"Point: {point_name}, Issue: {issue}")

except FileNotFoundError as e:
    print(f"Error: File not found. Check path: {input_excel_path}. Details: {str(e)}")
except Exception as e:
    print(f"Error: Failed to process file. Details: {str(e)}")

C:\Users\AAAli\AppData\Local\Temp\ipykernel_4584\6781253.py:48: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  updated_df = grouped.apply(update_point_group).reset_index(drop=True)


Updated Excel file saved to: Data_Updated_Depths_Total_Depth.xlsx
Original number of rows: 906
Updated number of rows: 906

Sample of Updated DataFrame:
      point_name  Depth_1  Depth_2 Type_Geo  Total_Depth  Thickness_meters  \
0  AbfaBazarReza     37.0     75.0   CL- ML    54.741631              38.0   
1  AbfaChenaran1     30.0     40.0       SM    59.647648              10.0   
2  AbfaChenaran1     40.0     50.0   CL- ML    59.647648              10.0   
3  AbfaChenaran1     50.0     55.0       SM    59.647648               5.0   
4  AbfaChenaran1     55.0     65.0       GW    59.647648              10.0   

   Thickness_Percent  Water_Level_Depth  
0          69.417004          54.741631  
1          16.765120          59.647648  
2          16.765120          59.647648  
3           8.382560          59.647648  
4          16.765120          59.647648  


In [6]:
import pandas as pd
input_excel_path = "Water_Level_1992_2024_Overlap.xlsx"
df = pd.read_excel(input_excel_path)
df

,point_name,Depth_1,Depth_2,Type_Geo,Elevation,Total_Depth,Lon_D,Lat_D,Thickness_meters,Thickness_Percent,Full_Name,Category,Thickness_Sum,Thickness_Difference,Thickness_Status,Water_Level_Depth,WL1992
0,abaskhan asma,5.0,10.0,CL,1170.0,90.0,59.228,36.6651,5.0,5.555556,Clay (Low Plasticity),Fine,100.0,0.0,0,23.229906,9.874941
1,abaskhan asma,10.0,20.0,GW,1170.0,90.0,59.228,36.6651,10.0,11.111111,Well-Graded Gravel,Coarse,100.0,0.0,0,23.229906,9.874941
2,abaskhan asma,20.0,25.0,CL,1170.0,90.0,59.228,36.6651,5.0,5.555556,Clay (Low Plasticity),Fine,100.0,0.0,0,23.229906,9.874941
3,abfaakharnakhrisi,22.0,70.0,GC,980.0,188.0,59.619,36.2625,48.0,25.531915,Gravelly Clay,Coarse,100.0,0.0,0,58.952065,47.912453
4,abfaandishe3,60.0,90.0,GC,1019.0,216.0,59.516,36.3807,30.0,13.888889,Gravelly Clay,Coarse,100.0,0.0,0,87.593704,69.350441
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
901,zanaghel asma,25.0,39.0,CL,1158.0,64.0,59.159,36.6447,14.0,21.875000,Clay (Low Plasticity),Fine,100.0,0.0,0,46.127895,25.860218
902,zanaghel asma,39.0,41.0,GP,1158.0,64.0,59.159,36.6447,2.0,3.125000,Poorly Graded Gravel,Coarse,100.0,0.0,0,46.127895,25.860218
903,zanaghel asma,41.0,45.0,CL,1158.0,64.0,59.159,36.6447,4.0,6.250000,Clay (Low Plasticity),Fine,100.0,0.0,0,46.127895,25.860218
904,zanaghel asma,45.0,46.0,GW,1158.0,64.0,59.159,36.6447,1.0,1.562500,Well-Graded Gravel,Coarse,100.0,0.0,0,46.127895,25.860218


In [7]:
import pandas as pd



input_excel_path = "Water_Level_1992_2024_Overlap.xlsx"
df = pd.read_excel(input_excel_path)
df
# Step 1: Sort by point_name and Depth_2 to ensure the last row is the deepest for each point
df = df.sort_values(by=['point_name', 'Depth_2'])

# Step 2: Update Depth_2 for the last row of each point_name with Water_Level_Depth
last_rows = df.groupby('point_name').tail(1).index
df.loc[last_rows, 'Depth_2'] = df.loc[last_rows, 'Water_Level_Depth']

# Step 3: Update Total_Depth for each point_name with Water_Level_Depth
df['Total_Depth'] = df.groupby('point_name')['Water_Level_Depth'].transform('first')

# Step 4: Sort back to original order to maintain the original row sequence
df = df.sort_index()

# Step 5: Save the updated DataFrame with all columns to a CSV file
df.to_csv('updated_data.csv', index=False)

# Step 6: Confirm the data has been saved and show the columns preserved
print("Data has been saved to 'updated_data.csv'")
print("Columns in the saved dataset:", df.columns.tolist())

Data has been saved to 'updated_data.csv'
Columns in the saved dataset: ['point_name', 'Depth_1', 'Depth_2', 'Type_Geo', 'Elevation', 'Total_Depth', 'Lon_D', 'Lat_D', 'Thickness_meters', 'Thickness_Percent', 'Full_Name', 'Category', 'Thickness_Sum', 'Thickness_Difference', 'Thickness_Status', 'Water_Level_Depth', 'WL1992']


In [8]:
df

,point_name,Depth_1,Depth_2,Type_Geo,Elevation,Total_Depth,Lon_D,Lat_D,Thickness_meters,Thickness_Percent,Full_Name,Category,Thickness_Sum,Thickness_Difference,Thickness_Status,Water_Level_Depth,WL1992
0,abaskhan asma,5.0,10.000000,CL,1170.0,23.229906,59.228,36.6651,5.0,5.555556,Clay (Low Plasticity),Fine,100.0,0.0,0,23.229906,9.874941
1,abaskhan asma,10.0,20.000000,GW,1170.0,23.229906,59.228,36.6651,10.0,11.111111,Well-Graded Gravel,Coarse,100.0,0.0,0,23.229906,9.874941
2,abaskhan asma,20.0,23.229906,CL,1170.0,23.229906,59.228,36.6651,5.0,5.555556,Clay (Low Plasticity),Fine,100.0,0.0,0,23.229906,9.874941
3,abfaakharnakhrisi,22.0,58.952065,GC,980.0,58.952065,59.619,36.2625,48.0,25.531915,Gravelly Clay,Coarse,100.0,0.0,0,58.952065,47.912453
4,abfaandishe3,60.0,87.593704,GC,1019.0,87.593704,59.516,36.3807,30.0,13.888889,Gravelly Clay,Coarse,100.0,0.0,0,87.593704,69.350441
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
901,zanaghel asma,25.0,39.000000,CL,1158.0,46.127895,59.159,36.6447,14.0,21.875000,Clay (Low Plasticity),Fine,100.0,0.0,0,46.127895,25.860218
902,zanaghel asma,39.0,41.000000,GP,1158.0,46.127895,59.159,36.6447,2.0,3.125000,Poorly Graded Gravel,Coarse,100.0,0.0,0,46.127895,25.860218
903,zanaghel asma,41.0,45.000000,CL,1158.0,46.127895,59.159,36.6447,4.0,6.250000,Clay (Low Plasticity),Fine,100.0,0.0,0,46.127895,25.860218
904,zanaghel asma,45.0,46.000000,GW,1158.0,46.127895,59.159,36.6447,1.0,1.562500,Well-Graded Gravel,Coarse,100.0,0.0,0,46.127895,25.860218


In [ ]:

input_excel_path = "Water_Level_1992_2024_Overlap.xlsx"
df = pd.read_excel(input_excel_path)
df

In [10]:
import pandas as pd

# Step 1: Load the dataset from updated_data.csv
df = pd.read_csv('updated_data.csv')

# Step 2: Recalculate Thickness_meters as Depth_2 - Depth_1
df['Thickness_meters'] = df['Depth_2'] - df['Depth_1']

# Step 3: Recalculate Thickness_Percent as (Thickness_meters / Total_Depth) * 100
df['Thickness_Percent'] = (df['Thickness_meters'] / df['Total_Depth']) * 100

# Step 4: Calculate percentage of fine, medium, and coarse layers for each point_name
# Group by point_name and Category, sum Thickness_meters
thickness_by_category = df.groupby(['point_name', 'Category'])['Thickness_meters'].sum().unstack(fill_value=0)

# Calculate percentage for each category relative to Total_Depth
# Get Total_Depth for each point_name (it's the same for all rows of a point_name)
total_depth = df.groupby('point_name')['Total_Depth'].first()
percentage_by_category = thickness_by_category.div(total_depth, axis=0) * 100
percentage_by_category = percentage_by_category.fillna(0).reset_index()

# Step 5: Calculate total thickness of fine layers in meters
fine_thickness = df[df['Category'] == 'Fine'].groupby('point_name')['Thickness_meters'].sum().reset_index()
fine_thickness.columns = ['point_name', 'Fine_Thickness_meters']

# Step 6: Save the percentage of fine, medium, and coarse layers to a CSV file
percentage_by_category.to_csv('fine_percentage.csv', index=False)

# Step 7: Save the total thickness of fine layers to a CSV file
fine_thickness.to_csv('fine_thickness.csv', index=False)

# Step 8: Save the updated main dataset with all 17 columns
df.to_csv('updated_data_final.csv', index=False)

# Step 9: Confirm outputs and show sample data
print("Updated main dataset saved to 'updated_data.csv'")
print("Percentage of fine, medium, and coarse layers saved to 'fine_percentage.csv'")
print("Total thickness of fine layers saved to 'fine_thickness.csv'")
print("\nColumns in updated dataset:", df.columns.tolist())
print("\nSample of updated main dataset:")
print(df[['point_name', 'Depth_1', 'Depth_2', 'Total_Depth', 'Thickness_meters', 'Thickness_Percent', 'Water_Level_Depth']].head(10))
print("\nSample of fine_percentage.csv:")
print(percentage_by_category.head())
print("\nSample of fine_thickness.csv:")
print(fine_thickness.head())

Updated main dataset saved to 'updated_data.csv'
Percentage of fine, medium, and coarse layers saved to 'fine_percentage.csv'
Total thickness of fine layers saved to 'fine_thickness.csv'

Columns in updated dataset: ['point_name', 'Depth_1', 'Depth_2', 'Type_Geo', 'Elevation', 'Total_Depth', 'Lon_D', 'Lat_D', 'Thickness_meters', 'Thickness_Percent', 'Full_Name', 'Category', 'Thickness_Sum', 'Thickness_Difference', 'Thickness_Status', 'Water_Level_Depth', 'WL1992']

Sample of updated main dataset:
          point_name  Depth_1    Depth_2  Total_Depth  Thickness_meters  \
0      abaskhan asma      5.0  10.000000    23.229906          5.000000   
1      abaskhan asma     10.0  20.000000    23.229906         10.000000   
2      abaskhan asma     20.0  23.229906    23.229906          3.229906   
3  abfaakharnakhrisi     22.0  58.952065    58.952065         36.952065   
4       abfaandishe3     60.0  87.593704    87.593704         27.593704   
5     abfaazadshahr2      0.0  65.000000    72.7

In [11]:
import pandas as pd

# Step 1: Load the dataset from updated_data.csv
df = pd.read_csv('updated_data.csv')

# Step 2: Recalculate Thickness_meters as Depth_2 - Depth_1
df['Thickness_meters'] = df['Depth_2'] - df['Depth_1']

# Step 3: Recalculate Thickness_Percent as (Thickness_meters / Total_Depth) * 100
df['Thickness_Percent'] = (df['Thickness_meters'] / df['Total_Depth']) * 100

# Step 4: Calculate percentage of fine, medium, and coarse layers for each point_name
# Group by point_name and Category, sum Thickness_meters
thickness_by_category = df.groupby(['point_name', 'Category'])['Thickness_meters'].sum().unstack(fill_value=0)

# Calculate percentage for each category relative to Total_Depth
# Get Total_Depth for each point_name (it's the same for all rows of a point_name)
total_depth = df.groupby('point_name')['Total_Depth'].first()
percentage_by_category = thickness_by_category.div(total_depth, axis=0) * 100
percentage_by_category = percentage_by_category.fillna(0).reset_index()

# Step 5: Calculate total thickness of fine layers in meters
fine_thickness = df[df['Category'] == 'Fine'].groupby('point_name')['Thickness_meters'].sum().reset_index()
fine_thickness.columns = ['point_name', 'Fine_Thickness_meters']

# Step 6: Get additional columns (Elevation, Lat_D, Lon_D) for each point_name
# Since these are constant per point_name, take the first value
additional_info = df.groupby('point_name').agg({
    'Elevation': 'first',
    'Total_Depth': 'first',
    'Lat_D': 'first',
    'Lon_D': 'first'
}).reset_index()

# Calculate Fine_Percentage for merging
fine_percentage = percentage_by_category[['point_name', 'Fine']].rename(columns={'Fine': 'Fine_Percentage'})

# Step 7: Merge additional columns into fine_percentage and fine_thickness
# Merge with percentage_by_category
percentage_by_category = percentage_by_category.merge(additional_info, on='point_name', how='left')
percentage_by_category = percentage_by_category.merge(fine_thickness, on='point_name', how='left')
percentage_by_category['Fine_Thickness_meters'] = percentage_by_category['Fine_Thickness_meters'].fillna(0)
percentage_by_category = percentage_by_category.merge(fine_percentage, on='point_name', how='left')

# Merge with fine_thickness
fine_thickness = fine_thickness.merge(additional_info, on='point_name', how='left')
fine_thickness = fine_thickness.merge(fine_percentage, on='point_name', how='left')

# Reorder columns for fine_percentage.csv
fine_percentage_cols = ['point_name', 'Fine', 'Medium', 'Coarse', 'Fine_Thickness_meters', 'Fine_Percentage', 'Elevation', 'Total_Depth', 'Lat_D', 'Lon_D']
percentage_by_category = percentage_by_category[fine_percentage_cols]

# Reorder columns for fine_thickness.csv
fine_thickness_cols = ['point_name', 'Fine_Thickness_meters', 'Fine_Percentage', 'Elevation', 'Total_Depth', 'Lat_D', 'Lon_D']
fine_thickness = fine_thickness[fine_thickness_cols]

# Step 8: Save the percentage of fine, medium, and coarse layers to a CSV file
percentage_by_category.to_csv('fine_percentage.csv', index=False)

# Step 9: Save the total thickness of fine layers to a CSV file
fine_thickness.to_csv('fine_thickness.csv', index=False)

# Step 10: Save the updated main dataset with all 17 columns
df.to_csv('updated_data_final.csv', index=False)

# Step 11: Confirm outputs and show sample data
print("Updated main dataset saved to 'updated_data.csv'")
print("Percentage of fine, medium, and coarse layers saved to 'fine_percentage.csv'")
print("Total thickness of fine layers saved to 'fine_thickness.csv'")
print("\nColumns in updated dataset:", df.columns.tolist())
print("\nSample of updated main dataset:")
print(df[['point_name', 'Depth_1', 'Depth_2', 'Total_Depth', 'Thickness_meters', 'Thickness_Percent', 'Water_Level_Depth']].head(10))
print("\nSample of fine_percentage.csv:")
print(percentage_by_category.head())
print("\nSample of fine_thickness.csv:")
print(fine_thickness.head())

Updated main dataset saved to 'updated_data.csv'
Percentage of fine, medium, and coarse layers saved to 'fine_percentage.csv'
Total thickness of fine layers saved to 'fine_thickness.csv'

Columns in updated dataset: ['point_name', 'Depth_1', 'Depth_2', 'Type_Geo', 'Elevation', 'Total_Depth', 'Lon_D', 'Lat_D', 'Thickness_meters', 'Thickness_Percent', 'Full_Name', 'Category', 'Thickness_Sum', 'Thickness_Difference', 'Thickness_Status', 'Water_Level_Depth', 'WL1992']

Sample of updated main dataset:
          point_name  Depth_1    Depth_2  Total_Depth  Thickness_meters  \
0      abaskhan asma      5.0  10.000000    23.229906          5.000000   
1      abaskhan asma     10.0  20.000000    23.229906         10.000000   
2      abaskhan asma     20.0  23.229906    23.229906          3.229906   
3  abfaakharnakhrisi     22.0  58.952065    58.952065         36.952065   
4       abfaandishe3     60.0  87.593704    87.593704         27.593704   
5     abfaazadshahr2      0.0  65.000000    72.7

In [14]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Step 1: Load the dataset from updated_data.csv
df = pd.read_csv('updated_data.csv')

# Step 2: Recalculate Thickness_meters as Depth_2 - Depth_1
df['Thickness_meters'] = df['Depth_2'] - df['Depth_1']

# Step 3: Recalculate Thickness_Percent as (Thickness_meters / Total_Depth) * 100
df['Thickness_Percent'] = (df['Thickness_meters'] / df['Total_Depth']) * 100

# Step 4: Calculate percentage of fine, medium, and coarse layers for each point_name
# Group by point_name and Category, sum Thickness_meters
thickness_by_category = df.groupby(['point_name', 'Category'])['Thickness_meters'].sum().unstack(fill_value=0)

# Calculate percentage for each category relative to Total_Depth
# Get Total_Depth for each point_name (it's the same for all rows of a point_name)
total_depth = df.groupby('point_name')['Total_Depth'].first()
percentage_by_category = thickness_by_category.div(total_depth, axis=0) * 100
percentage_by_category = percentage_by_category.fillna(0).reset_index()

# Step 5: Calculate total thickness of fine layers in meters
fine_thickness = df[df['Category'] == 'Fine'].groupby('point_name')['Thickness_meters'].sum().reset_index()
fine_thickness.columns = ['point_name', 'Fine_Thickness_meters']

# Step 6: Get additional columns (Elevation, Lat_D, Lon_D) for each point_name
# Since these are constant per point_name, take the first value
additional_info = df.groupby('point_name').agg({
    'Elevation': 'first',
    'Total_Depth': 'first',
    'Lat_D': 'first',
    'Lon_D': 'first'
}).reset_index()

# Calculate Fine_Percentage for merging
fine_percentage = percentage_by_category[['point_name', 'Fine']].rename(columns={'Fine': 'Fine_Percentage'})

# Step 7: Merge additional columns into fine_percentage and fine_thickness
# Merge with percentage_by_category
percentage_by_category = percentage_by_category.merge(additional_info, on='point_name', how='left')
percentage_by_category = percentage_by_category.merge(fine_thickness, on='point_name', how='left')
percentage_by_category['Fine_Thickness_meters'] = percentage_by_category['Fine_Thickness_meters'].fillna(0)
percentage_by_category = percentage_by_category.merge(fine_percentage, on='point_name', how='left')

# Merge with fine_thickness
fine_thickness = fine_thickness.merge(additional_info, on='point_name', how='left')
fine_thickness = fine_thickness.merge(fine_percentage, on='point_name', how='left')

# Reorder columns for fine_percentage.csv
fine_percentage_cols = ['point_name', 'Fine', 'Medium', 'Coarse', 'Fine_Thickness_meters', 'Fine_Percentage', 'Elevation', 'Total_Depth', 'Lat_D', 'Lon_D']
percentage_by_category = percentage_by_category[fine_percentage_cols]

# Reorder columns for fine_thickness.csv
fine_thickness_cols = ['point_name', 'Fine_Thickness_meters', 'Fine_Percentage', 'Elevation', 'Total_Depth', 'Lat_D', 'Lon_D']
fine_thickness = fine_thickness[fine_thickness_cols]

# Step 8: Save the percentage of fine, medium, and coarse layers to a CSV file
#percentage_by_category.to_csv('fine_percentage.csv', index=False)

# Step 9: Save the total thickness of fine layers to a CSV file
#fine_thickness.to_csv('fine_thickness.csv', index=False)

# Step 10: Save the updated main dataset with all 17 columns
#df.to_csv('updated_data_final.csv', index=False)

# Step 11: Analyze results with visualizations
# Bar chart for Fine, Medium, Coarse percentages
plt.figure(figsize=(12, 6))
percentage_by_category.set_index('point_name')[['Fine', 'Medium', 'Coarse']].plot(kind='bar', stacked=True, color=['#1f77b4', '#ff7f0e', '#2ca02c'])
plt.title('Percentage of Fine, Medium, and Coarse Layers by Point')
plt.xlabel('Point Name')
plt.ylabel('Percentage (%)')
plt.legend(title='Category')
plt.tight_layout()
plt.savefig('layer_percentage_bar.png')
plt.close()

# Pie charts for each point_name
for point in percentage_by_category['point_name']:
    point_data = percentage_by_category[percentage_by_category['point_name'] == point][['Fine', 'Medium', 'Coarse']].iloc[0]
    labels = ['Fine', 'Medium', 'Coarse']
    sizes = [point_data['Fine'], point_data['Medium'], point_data['Coarse']]
    # Only include non-zero values in the pie chart
    non_zero_labels = [label for label, size in zip(labels, sizes) if size > 0]
    non_zero_sizes = [size for size in sizes if size > 0]
    if non_zero_sizes:  # Only create pie chart if there are non-zero values
        plt.figure(figsize=(6, 6))
        plt.pie(non_zero_sizes, labels=non_zero_labels, autopct='%1.1f%%', colors=['#1f77b4', '#ff7f0e', '#2ca02c'])
        plt.title(f'Layer Distribution for {point}')
        plt.tight_layout()
        # Replace invalid characters in point_name for filename
        safe_point_name = point.replace(' ', '_').replace('/', '_').replace('\\', '_')
        plt.savefig(f'pie_chart_{safe_point_name}.png')
        plt.close()

# Histogram of Fine percentages
plt.figure(figsize=(10, 6))
sns.histplot(percentage_by_category['Fine'], bins=20, color='#1f77b4', kde=True)
plt.title('Distribution of Fine Layer Percentages')
plt.xlabel('Fine Percentage (%)')
plt.ylabel('Frequency')
plt.tight_layout()
plt.savefig('fine_percentage_histogram.png')
plt.close()

# Scatter plot of Fine_Thickness_meters vs Total_Depth with Elevation as color
plt.figure(figsize=(10, 6))
scatter = plt.scatter(percentage_by_category['Total_Depth'], percentage_by_category['Fine_Thickness_meters'], 
                     c=percentage_by_category['Elevation'], cmap='viridis', s=100)
plt.colorbar(scatter, label='Elevation (m)')
plt.title('Fine Thickness vs Total Depth')
plt.xlabel('Total Depth (m)')
plt.ylabel('Fine Thickness (m)')
plt.tight_layout()
plt.savefig('fine_thickness_vs_total_depth.png')
plt.close()

# Summary statistics for Fine, Medium, Coarse percentages
summary_stats = percentage_by_category[['Fine', 'Medium', 'Coarse']].describe()
summary_stats = summary_stats.loc[['mean', '50%', 'min', 'max']]

# Step 12: Confirm outputs and show sample data
print("Updated main dataset saved to 'updated_data_final.csv'")
print("Percentage of fine, medium, and coarse layers saved to 'fine_percentage.csv'")
print("Total thickness of fine layers saved to 'fine_thickness.csv'")
print("Bar chart saved to 'layer_percentage_bar.png'")
print("Pie charts saved as 'pie_chart_<point_name>.png' for each point")
print("Histogram of Fine percentages saved to 'fine_percentage_histogram.png'")
print("Scatter plot of Fine Thickness vs Total Depth saved to 'fine_thickness_vs_total_depth.png'")
print("\nColumns in updated dataset:", df.columns.tolist())
print("\nSample of updated main dataset:")
print(df[['point_name', 'Depth_1', 'Depth_2', 'Total_Depth', 'Thickness_meters', 'Thickness_Percent', 'Water_Level_Depth']].head(10))
print("\nSample of fine_percentage.csv:")
print(percentage_by_category.head())
print("\nSample of fine_thickness.csv:")
print(fine_thickness.head())
print("\nSummary statistics for Fine, Medium, Coarse percentages:")
print(summary_stats)

Updated main dataset saved to 'updated_data_final.csv'
Percentage of fine, medium, and coarse layers saved to 'fine_percentage.csv'
Total thickness of fine layers saved to 'fine_thickness.csv'
Bar chart saved to 'layer_percentage_bar.png'
Pie charts saved as 'pie_chart_<point_name>.png' for each point
Histogram of Fine percentages saved to 'fine_percentage_histogram.png'
Scatter plot of Fine Thickness vs Total Depth saved to 'fine_thickness_vs_total_depth.png'

Columns in updated dataset: ['point_name', 'Depth_1', 'Depth_2', 'Type_Geo', 'Elevation', 'Total_Depth', 'Lon_D', 'Lat_D', 'Thickness_meters', 'Thickness_Percent', 'Full_Name', 'Category', 'Thickness_Sum', 'Thickness_Difference', 'Thickness_Status', 'Water_Level_Depth', 'WL1992']

Sample of updated main dataset:
          point_name  Depth_1    Depth_2  Total_Depth  Thickness_meters  \
0      abaskhan asma      5.0  10.000000    23.229906          5.000000   
1      abaskhan asma     10.0  20.000000    23.229906         10.00000

<Figure size 1200x600 with 0 Axes>